In [1]:
import numpy as np
import pandas as pd
from mord import OrdinalRidge

In [2]:
x_ctype_train_file = "/home/bram/jointomicscomp/data/BRCA/BRCA_GE_train.npy"
y_ctype_train_file = "/home/bram/jointomicscomp/data/BRCA/BRCA_ME_train.npy"
x_ctype_valid_file = "/home/bram/jointomicscomp/data/BRCA/BRCA_GE_valid.npy"
y_ctype_valid_file = "/home/bram/jointomicscomp/data/BRCA/BRCA_ME_valid.npy"
x_ctype_test_file = "/home/bram/jointomicscomp/data/BRCA/BRCA_GE_test.npy"
y_ctype_test_file = "/home/bram/jointomicscomp/data/BRCA/BRCA_ME_test.npy"

# For latent feature extraction
GEtrainctype = np.load(x_ctype_train_file)
GEvalidctype = np.load(x_ctype_valid_file)
GEtestctype = np.load(x_ctype_test_file)

In [3]:
y_train = np.load("/home/bram/jointomicscomp/data/BRCA/BRCA_train_stageType.npy")
y_valid = np.load("/home/bram/jointomicscomp/data/BRCA/BRCA_valid_stageType.npy")
y_test = np.load("/home/bram/jointomicscomp/data/BRCA/BRCA_test_stageType.npy")



In [4]:
# Z1 for GE and z2 for ME
z_moe = np.load("/home/bram/jointomicscomp/results/trained MVAE task 2/MoE/task2_z.npy")
z_poe = np.load("//home/bram/jointomicscomp/results/trained MVAE task 2/PoE/task2_z.npy")

In [5]:
# Create labels from stageTypes and stageTypes
latent_train1 = z_moe[:len(GEtrainctype)]
latent_valid1 = z_moe[len(latent_train1):(len(GEtrainctype) + len(GEvalidctype))]
latent_test1 = z_moe[(len(latent_train1) + len(latent_valid1)):(len(GEtrainctype) + len(GEvalidctype) + len(GEtestctype))]

print(latent_train1.shape)
print(latent_valid1.shape)
print(latent_test1.shape)

latent_train2 = z_poe[:len(GEtrainctype)]
latent_valid2 = z_poe[len(latent_train2):(len(GEtrainctype) + len(GEvalidctype))]
latent_test2 = z_poe[(len(latent_train2) + len(latent_valid2)):(len(GEtrainctype) + len(GEvalidctype) + len(GEtestctype))]

print(latent_train2.shape)
print(latent_valid2.shape)
print(latent_test2.shape)

(687, 128)
(77, 128)
(85, 128)
(687, 128)
(77, 128)
(85, 128)


In [6]:
np.concatenate((y_train, y_valid)).shape

(764,)

In [7]:
print("Accuracies from Mixture-of-Experts")
accuracies = []
alphas = np.array([1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1.0, 2.0, 5.0, 10., 20.])
for i, a in enumerate(alphas):
    model = OrdinalRidge(alpha=a, fit_intercept=True, normalize=False, random_state=1)

    # Train
    model.fit(np.vstack((latent_train1, latent_valid1)), np.concatenate((y_train, y_valid)))

    y_pred = model.predict(latent_test1)

    n = len(y_pred)
    correct = 0

    for t, p in zip(y_test, y_pred):
        if t == p:
            correct += 1

    accuracy = (correct / n) * 100
    accuracies.append(round(accuracy, 2))

print("Accuracies: ", accuracies)


Accuracies from Mixture-of-Experts
Accuracies:  [54.12, 54.12, 55.29, 54.12, 57.65, 57.65, 57.65, 57.65, 57.65, 57.65]


In [8]:
y_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)

In [9]:
print("Accuracies from Product-of-Experts")
accuracies = []
alphas = np.array([1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1.0, 2.0, 5.0, 10., 20.])
for i, a in enumerate(alphas):
    model = OrdinalRidge(alpha=a, fit_intercept=True, normalize=False, random_state=1)

    # Train
    model.fit(np.vstack((latent_train2, latent_valid2)), np.concatenate((y_train, y_valid)))

    y_pred = model.predict(latent_test2)

    n = len(y_pred)
    correct = 0

    for t, p in zip(y_test, y_pred):
        if t == p:
            correct += 1

    accuracy = (correct / n) * 100
    accuracies.append(round(accuracy, 2))

print("Accuracies: ", accuracies)


Accuracies from Product-of-Experts
Accuracies:  [51.76, 52.94, 58.82, 57.65, 57.65, 57.65, 57.65, 57.65, 57.65, 57.65]
